In [1]:
import ikea_api
import numpy as np
import pandas as pd
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from tqdm.autonotebook import tqdm
import time
import joblib
from joblib import Parallel, delayed
from pprint import pprint
tqdm.pandas()

C:\Users\nanrc\AppData\Local\Temp\ipykernel_34520\1477617386.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
options = Options()
options.add_argument('--headless')
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
constants = ikea_api.Constants(country = 'us', language = 'en')
search = ikea_api.Search(constants)

In [4]:
lists_to_find = ['chair', 'table', 'umbrella', 'sofa']
limit = 20
result_lists = []
for find in tqdm(lists_to_find) :
    endpoint = search.search(find, limit = limit)
    result_lists.append(ikea_api.run(endpoint))
print(result_lists)

  0%|          | 0/4 [00:00<?, ?it/s]

[{'usergroup': '[]', 'testActivationTriggers': {}, 'searchResultPage': {'searchPhrase': 'chair', 'storeWindow': [], 'storeCount': 0, 'didYouMean': [], 'relatedSearches': [{'count': 140, 'text': 'desk chair', 'metadata': '1189'}, {'count': 145, 'text': 'office chair', 'metadata': '1065'}, {'count': 475, 'text': 'armchair', 'metadata': '486'}, {'count': 111, 'text': 'stool', 'metadata': '469'}, {'count': 423, 'text': 'table', 'metadata': '425'}, {'count': 408, 'text': 'dining chair', 'metadata': '390'}, {'count': 894, 'text': 'chair accent', 'metadata': '294'}, {'count': 161, 'text': 'swivel chair', 'metadata': '294'}, {'count': 1508, 'text': 'sofa', 'metadata': '292'}, {'count': 313, 'text': 'rug', 'metadata': '282'}], 'retiredProducts': [], 'products': {'badge': 1324, 'filters': [{'id': 'CATEGORIES', 'name': 'Category', 'collapsed': False, 'type': 'CATEGORY_TREE', 'parameter': 'f-subcategories', 'eventAction': 'filter_by_category', 'enabled': True, 'selected': False, 'actionTokens': []

In [5]:
def iterate_product(prod) :
    result_df = pd.DataFrame(columns = ['name', 'type', 'ratingValue', 'ratingCount', 'price', 'url', 'imgUrl', 'imgAlt'])
    idx_prod = lists_to_find.index(prod.lower())
    for idx, res in enumerate(result_lists[idx_prod]['searchResultPage']['products']['main']['items']) :
        res = res['product']
        try :
            res['ratingCount']
        except KeyError :
            res['ratingCount'] = 0
            res['ratingValue'] = 0
        res_df = pd.DataFrame(columns = ['name', 'type', 'ratingValue', 'ratingCount', 'price', 'url', 'imgUrl', 'imgAlt'])
        res_df['name'] = [res['name']]
        res_df['type'] = [res['typeName']]
        res_df['ratingValue'] = [res['ratingValue']]
        res_df['ratingCount'] = [res['ratingCount']]
        res_df['price'] = [f"{res['salesPrice']['numeral']} {res['salesPrice']['currencyCode']}"]
        res_df['url'] = [res['pipUrl']]
        res_df['imgUrl'] = [res['mainImageUrl']]
        res_df['imgAlt'] = [res['mainImageAlt']]
        if res['gprDescription']['numberOfVariants'] < 1 : continue
        for variant in res['gprDescription']['variants'] :
            try :
                variant['ratingCount']
            except KeyError :
                variant['ratingCount'] = 0
                variant['ratingValue'] = 0
            variant_df = pd.DataFrame(columns = ['name', 'type', 'ratingValue', 'ratingCount', 'price', 'url', 'imgUrl', 'imgAlt'])
            variant_df['url'] = [variant['pipUrl']]
            variant_df['imgUrl'] = [variant['imageUrl']]
            variant_df['imgAlt'] = [variant['imageAlt']]
            variant_df['ratingValue'] = [variant['ratingValue']]
            variant_df['ratingCount'] = [variant['ratingCount']]
            variant_df['name'] = [variant['name']]
            variant_df['type'] = [variant['typeName']]
            variant_df['price'] = [f"{variant['salesPrice']['numeral']} {variant['salesPrice']['currencyCode']}"]
            res_df = pd.concat([res_df, variant_df], axis = 0, ignore_index = True)
        result_df = pd.concat([result_df, res_df], axis = 0, ignore_index = True)
    return result_df.reset_index(drop = True)

In [6]:
chair_df = iterate_product('chair')
table_df = iterate_product('table')
umbrella_df = iterate_product('umbrella')
sofa_df = iterate_product('sofa')

C:\Users\nanrc\AppData\Local\Temp\ipykernel_34520\1319726111.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, res_df], axis = 0, ignore_index = True)
C:\Users\nanrc\AppData\Local\Temp\ipykernel_34520\1319726111.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, res_df], axis = 0, ignore_index = True)
C:\Users\nanrc\AppData\Local\Temp\ipykernel_34520\1319726111.py:37: FutureWarning: The behavior of DataFrame concatenation w

In [7]:
all_df = pd.concat([
    chair_df,
    table_df,
    umbrella_df,
    sofa_df
], axis = 0, ignore_index = True)
all_df

,name,type,ratingValue,ratingCount,price,url,imgUrl,imgAlt
0,ADDE,Chair,4.5,651,15.0 USD,https://www.ikea.com/us/en/p/adde-chair-white-...,https://www.ikea.com/us/en/images/products/add...,"ADDE Chair, white"
1,ADDE,Chair,4.5,651,15.0 USD,https://www.ikea.com/us/en/p/adde-chair-black-...,https://www.ikea.com/us/en/images/products/add...,"ADDE Chair, black"
2,POÄNG,Armchair,4.7,318,119.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Knisa light beige"
3,POÄNG,Armchair,4.7,318,189.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Hillared beige"
4,POÄNG,Armchair,4.7,318,189.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Hillared anthracite"
...,...,...,...,...,...,...,...,...
252,UPPLAND,Loveseat,4.4,108,849.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Hillared dark blue"
253,UPPLAND,Loveseat,4.4,108,799.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Kelinge gray-turquoise"
254,UPPLAND,Loveseat,4.4,108,699.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Totebo dark turquoise"
255,UPPLAND,Loveseat,4.4,108,749.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Blekinge white"


In [8]:
print(all_df['url'][2])

https://www.ikea.com/us/en/p/poaeng-armchair-birch-veneer-knisa-light-beige-s59305928/


In [9]:
options = Options()
# options.add_argument('--headless')
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [10]:
def scrape_ikea_desc(url, driver = driver) :
    dsc = ''
    driver.get(url)
    elements = driver.find_elements(By.XPATH, '//p[contains(@class, "pip-product-details__paragraph")]')
    for element in elements :
        dsc += element.get_attribute('innerHTML') + '\n'
    return dsc

In [11]:
all_df['description'] = [scrape_ikea_desc(url) for url in tqdm(all_df['url'])]
display(all_df)

  0%|          | 0/257 [00:00<?, ?it/s]

,name,type,ratingValue,ratingCount,price,url,imgUrl,imgAlt,description
0,ADDE,Chair,4.5,651,15.0 USD,https://www.ikea.com/us/en/p/adde-chair-white-...,https://www.ikea.com/us/en/images/products/add...,"ADDE Chair, white","You can stack the chairs, so they take less sp..."
1,ADDE,Chair,4.5,651,15.0 USD,https://www.ikea.com/us/en/p/adde-chair-black-...,https://www.ikea.com/us/en/images/products/add...,"ADDE Chair, black",Recommended for indoor use only.\nTo prevent s...
2,POÄNG,Armchair,4.7,318,119.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Knisa light beige",The layer-glued bent wood frame gives the armc...
3,POÄNG,Armchair,4.7,318,189.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Hillared beige",The layer-glued bent wood frame gives the armc...
4,POÄNG,Armchair,4.7,318,189.0 USD,https://www.ikea.com/us/en/p/poaeng-armchair-b...,https://www.ikea.com/us/en/images/products/poa...,"POÄNG Armchair, birch veneer/Hillared anthracite",The layer-glued bent wood frame gives the armc...
...,...,...,...,...,...,...,...,...,...
252,UPPLAND,Loveseat,4.4,108,849.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Hillared dark blue",Enjoy the super comfy UPPLAND sofa with embrac...
253,UPPLAND,Loveseat,4.4,108,799.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Kelinge gray-turquoise",Enjoy the super comfy UPPLAND sofa with embrac...
254,UPPLAND,Loveseat,4.4,108,699.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Totebo dark turquoise",Enjoy the super comfy UPPLAND sofa with embrac...
255,UPPLAND,Loveseat,4.4,108,749.0 USD,https://www.ikea.com/us/en/p/uppland-loveseat-...,https://www.ikea.com/us/en/images/products/upp...,"UPPLAND Loveseat, Blekinge white",Enjoy the super comfy UPPLAND sofa with embrac...


In [12]:
all_df.to_csv('ikea_ds.tsv', sep = '\t', index = False)